### The logic of the notebook

In order to be able to perform multiple operations I have designed the system as follows:

- first, I create the most basic function which just updates the owner of a player
- then I defined the class transfer, with argument name of the player
    - the class transfer can perform all market operations and then save database entries
    - however, it is not convenient to save directly to the database, in case there are accidental mistakes
- so I created another class called market session:
    - first, you start a market session and create each transfer within the session using the inner class transfer
    - when you are done with all the transfers, and are sure there are no mistakes you confirm the session
    - if you notice a mistake you can cancel the session and the databases rest unchanged
    
    
## OPEN QUESTIONS:

- what is the cost for draft? I set it to 0 for now

In [2]:
import pandas as pd
import numpy as np
import datetime
from datetime import date
import json
import copy
from pymongo import MongoClient
from pprint import pprint
import pymongo

from datetime import date

with open('credential.json','r') as f:
    cred = json.load(f)

cluster = MongoClient(cred['cred'])
# choosing database
db = cluster["Game"]
# choosing collection
collection = db["Players"]

/Users/lucaagozzino/opt/anaconda3/lib/python3.7/site-packages/pymongo/common.py:559: UserWarning: Unknown option retryWrites
  warnings.warn(str(exc))


In [3]:
collection.find_one({'name': 'MUSSO'})

{'_id': 2792,
 'name': 'MUSSO',
 'info': {'personal_info': {'full_name': 'Musso Juan',
   'birthdate': '06/05/1994',
   'nation': 'Argentina, Italia',
   'team_real': 'Udinese',
   'FC_role': 'P'},
  'contract': {'owner': 'pietro',
   'start_date': '01/08/2020',
   'cost': 0,
   'acquisition_mode': '',
   'previous_owner': '',
   'quotation_initial': 11},
  'current_team': {'owner': 'pietro',
   'squad': 'main',
   'start_date': '01/08/2020',
   'previous_team': '',
   'quotation_initial': 11,
   'on_loan': False},
  'stats': {'Pg': 4,
   'Mv': 5.88,
   'Mf': 4.38,
   'Gf': 0,
   'Gs': 6,
   'Rp': 0,
   'Rc': 0,
   'R+': 0,
   'R-': 0,
   'Ass': 0,
   'Asf': 0,
   'Amm': 0,
   'Esp': 0,
   'Au': 0}}}

In [4]:
"""
Function documentation:
This is the general function to update the owner of a player.
Arguments:

- new_owner: string, the new owner of the team (pietro, luca,...)
- mode: string, the acquisition mode (Scambio, draft,...)
- squad: assignment in the new team (main or primavera)
- name/id: string, use one OR the other as a filter to query the database
- time_stamp: boolean. True if you want to register the date of operation in the database
- rest: details to access database, set to default values

Note: I'm planning to use this function inside higher level functions, which is why I have the timestamp option, 
if you are doing tests there is no reason to update the date, but if you include this in a market operation function
(for instance exchange players) you want to register this and the higher level function will also create the database entry

"""

def update_owner(new_owner, mode, squad, loan = False, move_up = False, name = False, Id = False, time_stamp = False, CR = cred['cred'], DB = 'Game', CO = 'Players'):
    
    cluster = MongoClient(CR)
    db = cluster[DB]
    collection = db[CO]
    
    today = date.today()
    d1 = today.strftime("%Y/%m/%d")
    
    if name:
        filt = {'name': name}
    elif Id:
        filt = {'_id': str(Id)}
    else:
        print('Please enter name or Id argument')
        
        return None
        
    dic = collection.find_one(filt)
    old_owner = dic['info']['contract']['owner'] #when database structure changes, Modify this and following accordingly
    old_team = dic['info']['current_team']['owner'] +', ' + dic['info']['current_team']['squad']
    
    if loan:

        collection.update_one(filt, {'$set':{'info.current_team.previous_team': old_team}}) 
        collection.update_one(filt, {'$set':{'info.current_team.owner': new_owner}})
        collection.update_one(filt, {'$set':{'info.current_team.squad': squad}})
        
        
        if time_stamp:
            collection.update_one(filt, {'$set':{'info.current_team.start_date': d1}}) #
            print("Operation date: ")
            return d1
    
    elif move_up:
        
        collection.update_one(filt, {'$set':{'info.current_team.previous_team': old_team}}) 
        collection.update_one(filt, {'$set':{'info.current_team.squad': squad}})
        
        if time_stamp:
            collection.update_one(filt, {'$set':{'info.current_team.start_date': d1}}) #
            print("Operation date: ")
            return d1
        
        
    else:
    
        collection.update_one(filt, {'$set':{'info.contract.acquisition_mode': mode}})
        collection.update_one(filt, {'$set':{'info.contract.previous_owner': old_owner}})
        collection.update_one(filt, {'$set':{'info.contract.owner': new_owner}})

        collection.update_one(filt, {'$set':{'info.current_team.previous_team': old_team}}) 
        collection.update_one(filt, {'$set':{'info.current_team.owner': new_owner}})
        collection.update_one(filt, {'$set':{'info.current_team.squad': squad}})
    
        if time_stamp:
            collection.update_one(filt, {'$set':{'info.contract.start_date': d1}})
            collection.update_one(filt, {'$set':{'info.current_team.start_date': d1}}) #redundant
            print("Operation date: ")
            return d1

In [26]:
"""
This is an inner level class, it performs all the market operations and creates database entries
But we'll use it inside another class, so all operations need to be confirmed before being saved

"""


class transfer:
    
    #need to add quotation info everywhere, maybe as return of transf_db

    def __init__(self, name, CR, DB, CO):
        
        cluster = MongoClient(CR)
        db = cluster[DB]
        
        self.collection = db[CO]
        self.name = name
        self.info = collection.find_one({'name': name})
        self.CC = (CR, DB, CO)
    
    def draft_in(self, new_owner, squad, cost = 0):
        (CR, DB, CO) = self.CC
        #print(new_owner+' has acquired '+self.name+' through draft.\nAssigned to '+squad+' squad\n')
        d1 = update_owner(new_owner = new_owner, mode = 'Draft', squad = squad, name = self.name, time_stamp = True, CR = CR, DB = DB, CO = CO)
        
        self.collection.update_one({'name': self.name},{'$set':{'info.contract.cost': cost}})
        #transf_db_update() #function to be defined
        
        return d1
    
    def auction_in(self, new_owner, squad, cost):
        (CR, DB, CO) = self.CC
        #print(new_owner+' has acquired '+self.name+' through auction for '+str(cost)+' FM.\nAssigned to '+squad+' squad\n')
        d1 = update_owner(new_owner = new_owner, mode = 'Asta', squad = squad, name = self.name, time_stamp = True, CR = CR, DB = DB, CO = CO)
        
        self.collection.update_one({'name': self.name},{'$set':{'info.contract.cost': cost}})
        #transf_db_update() #function to be defined, includes budgets adjustments
        
        return d1
    
    def cash_in(self, new_owner, squad, cost):
        (CR, DB, CO) = self.CC
        #print(new_owner+' has acquired '+self.name+' through auction for '+str(cost)+' FM.\nAssigned to '+squad+' squad\n')
        d1 = update_owner(new_owner = new_owner, mode = 'Acquisto', squad = squad, name = self.name, time_stamp = True, CR = CR, DB = DB, CO = CO)
        
        self.collection.update_one({'name': self.name},{'$set':{'info.contract.cost': cost}})
        #transf_db_update() #function to be defined, includes budgets adjustments
        
        
        return d1
    
    def exchange_with(self, exch_player, cost, squad, squad_exch):
        
        #PROBABLY SOME MISTAKE HERE, DOUBLE CHECK
        
        (CR, DB, CO) = self.CC
        collection = self.collection
        self.exch_player = exch_player
        
        
        exch_info = collection.find_one({'name': exch_player})
        exch_owner = exch_info['info']['contract']['owner']
        owner = self.info['info']['contract']['owner']
        
        
        d1 = update_owner(new_owner = exch_owner, mode = 'Scambio', squad = squad, name = self.name, time_stamp = False, CR = CR, DB = DB, CO = CO)
        d1 = update_owner(new_owner = owner, mode = 'Scambio', squad = squad_exch, name = exch_player, time_stamp = True, CR = CR, DB = DB, CO = CO)
        
        self.collection.update_one({'name': self.name},{'$set':{'info.contract.cost': max(0, cost)}})
        self.collection.update_one({'name': exch_player},{'$set':{'info.contract.cost': max(0, -cost)}})
        
        #transf_db_update() #function to be defined, includes budgets adjustments
        return d1
    
    def loan_to(self, owner, cost, squad, end_season = None, final_fee = None):
        (CR, DB, CO) = self.CC
        collection = self.collection
        d1 = update_owner(new_owner = owner, mode = None, squad = squad, loan = True, name = self.name, time_stamp = True, CR = CR, DB = DB, CO = CO)
        
        import datetime
        date = datetime.datetime.strptime(d1, '%Y/%m/%d')
        
        m, y = date_obj.month, date_obj.year
        if m > y:
            y = y + 1
        
        loan_info = {
        'start_date': d1,
        'expire_date': str(y)+'/07/31',
        'cost': cost,
        'redemption_info': end_season, #None, Option or Must
        'redemption_cost': final_fee, # None or >0
        }
        
        collection.update_one({'name': self.name}, {'$set':{'info.current_team.on_loan': True}}) 
        collection.update_one({'name': self.name}, {'$set':{'info.current_team.loan_info': loan_info}}) 
        
        return d1
    
    def end_loan(self, squad):
        (CR, DB, CO) = self.CC
        collection = self.collection
        
        dic = self.info
        owner = dic['info']['contract']['owner']
        
        d1 = update_owner(new_owner = owner, mode = None, squad = squad, loan = True, name = self.name, time_stamp = True, CR = CR, DB = DB, CO = CO)
        
        collection.update_one({'name': self.name}, {'$set':{'info.current_team.on_loan': False}}) 
        collection.update_one({'name': self.name}, {'$set':{'info.current_team.loan_info': None}}) 
        
        return d1
        
        
    def move_up(self):
        (CR, DB, CO) = self.CC
        d1 = update_owner(new_owner = None, mode = None, squad = 'main', move_up = True, name = self.name, time_stamp = True, CR = CR, DB = DB, CO = CO)
        
        return d1
        
        

In [27]:
"""
This second layer is designed to prevent accidental changes in the database:
First all the players collection is duplicated to a temp one
then all the changes are made in the temp

NOTE: we need to add the equivalent for the transfer database, but we first need to design the db_update function
"""

class market_session:
    
    def __init__(self, ID, CR = cred['cred'], DB = 'Game', CO = 'Players'):
        #copy entire database into tempPlayers
        
        cluster = MongoClient(CR)
        db = cluster[DB]
        collection = db[CO]
        COt = 'tempPlayers'
        collectionT = db[COt]
        
        posts = list(collection.find({}))
        collectionT.insert_many(posts)
        
        self.ID = ID
        self.CC = (CR, DB, COt)
        self.CO = CO
        
        
    def select_pl(self, name):
        (CR, DB, COt) = self.CC
        self.player = transfer(name, CR, DB, COt)
    
    def confirm_session(self):
        #here we span through both temp collections (players and transfers) and replace/add entries to the official database
        #then we erase the temp databases
        (CR, DB, COt) = self.CC
        CO = self.CO
        
        cluster = MongoClient(CR)
        db = cluster[DB]
        collection = db[CO]
        
        collectionT = db[COt]
        posts = list(collectionT.find({}))
        collection.delete_many({})
        collection.insert_many(posts)
        
        collectionT.delete_many({})
        
        
    def cancel_session(self):
        #erase temporary database: need to redefine market session
        (CR, DB, COt) = self.CC
        
        cluster = MongoClient(CR)
        db = cluster[DB]
        collectionT = db[COt]
        
        collectionT.delete_many({})

In [28]:
def player(name, session):
    #select one player and assign it to a variable 
    session.select_pl(name)
    return session.player #returns a class transfer object

In [30]:
#define market session with id number (not necessary)
sess = market_session(1)

In [31]:
#create a transfer class object
pl1 = player('MUSSO', sess)

In [66]:
#another example
pl2 = player('RONALDO', sess)

In [67]:
#check name
pl2.name

'RONALDO'

In [32]:
#perform operations
pl1.draft_in('luca','primavera')

Operation date: 


'2020/12/22'

In [82]:
pl2.loan_to('luca', 10, 'primavera', 'Option', final_fee = 100)

Operation date: 


In [33]:
#check transaction
db['tempPlayers'].find_one({'name': 'MUSSO'})

{'_id': 2792,
 'name': 'MUSSO',
 'info': {'personal_info': {'full_name': 'Musso Juan',
   'birthdate': '06/05/1994',
   'nation': 'Argentina, Italia',
   'team_real': 'Udinese',
   'FC_role': 'P'},
  'contract': {'owner': 'luca',
   'start_date': '2020/12/22',
   'cost': 100,
   'acquisition_mode': 'Draft',
   'previous_owner': 'pietro',
   'quotation_initial': 11},
  'current_team': {'owner': 'luca',
   'squad': 'primavera',
   'start_date': '2020/12/22',
   'previous_team': 'pietro, main',
   'quotation_initial': 11,
   'on_loan': False},
  'stats': {'Pg': 4,
   'Mv': 5.88,
   'Mf': 4.38,
   'Gf': 0,
   'Gs': 6,
   'Rp': 0,
   'Rc': 0,
   'R+': 0,
   'R-': 0,
   'Ass': 0,
   'Asf': 0,
   'Amm': 0,
   'Esp': 0,
   'Au': 0}}}

In [29]:
#UNCOMMENT ONLY AT THE END OF THE SESSION WHEN YOU ARE SURE EVERYTHING IS CORRECT

#sess.confirm_session()

#UNCOMMENT IF YOU WANT TO REVERT ALL THE OPERATIONS OF CURRENT SESSION

sess.cancel_session()

In [93]:
collection.find_one({'name': 'MUSSO'})

{'_id': 2792,
 'name': 'MUSSO',
 'info': {'personal_info': {'full_name': 'Musso Juan',
   'birthdate': '06/05/1994',
   'nation': 'Argentina, Italia',
   'team_real': 'Udinese',
   'FC_role': 'P'},
  'contract': {'owner': 'pietro',
   'start_date': '01/08/2020',
   'cost': 0,
   'acquisition_mode': '',
   'previous_owner': '',
   'quotation_initial': 11},
  'current_team': {'owner': 'pietro',
   'squad': '',
   'start_date': '01/08/2020',
   'previous_team': '',
   'quotation_initial': 11,
   'on_loan': False},
  'stats': {'Pg': 4,
   'Mv': 5.88,
   'Mf': 4.38,
   'Gf': 0,
   'Gs': 6,
   'Rp': 0,
   'Rc': 0,
   'R+': 0,
   'R-': 0,
   'Ass': 0,
   'Asf': 0,
   'Amm': 0,
   'Esp': 0,
   'Au': 0}}}